In [1]:
# Import Dependencies
import pandas as pd
import requests
import gmaps

In [2]:
# Import API key
from config import g_key

In [3]:
# Configure gmaps api key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Vardo,NO,70.3705,31.1107,37.38,100,100,12.66,overcast clouds
1,1,Torbay,CA,47.6666,-52.7314,47.17,57,20,18.41,few clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,71.19,78,5,4.79,clear sky
3,3,Pachino,IT,36.7186,15.0907,60.26,67,0,3.02,clear sky
4,4,Vila Franca Do Campo,PT,37.7167,-25.4333,66.20,92,100,3.96,overcast clouds


In [5]:
# 2. Prompt the use to enter a minimum and maximum temp criteria
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]


In [7]:
# 4a. Determine if there are any empty rows
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 11 to 717
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_ID      185 non-null    int64  
 1   City         185 non-null    object 
 2   Country      185 non-null    object 
 3   Lat          185 non-null    float64
 4   Lng          185 non-null    float64
 5   Max Temp     185 non-null    float64
 6   Humidity     185 non-null    int64  
 7   Cloudiness   185 non-null    int64  
 8   Wind Speed   185 non-null    float64
 9   Description  185 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 15.9+ KB


In [8]:
# 4b. Drop any any empty rows and create a new DataFrame

In [9]:
# 5a. Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, current description, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Humidity", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Humidity,Max Temp,Description,Lat,Lng,Hotel Name
11,Victoria,HK,95,78.85,moderate rain,22.2855,114.1577,
14,Oranjestad,AW,78,82.35,few clouds,12.5240,-70.0270,
18,La Asuncion,VE,74,80.33,broken clouds,11.0333,-63.8628,
30,Georgetown,MY,89,75.15,light rain,5.4112,100.3354,
34,Grand Gaube,MU,80,75.38,scattered clouds,-20.0064,57.6608,
39,Saint-Louis,SN,48,87.44,clear sky,16.3333,-15.0000,
40,Mahebourg,MU,94,75.25,few clouds,-20.4081,57.7000,
49,Caraballeda,VE,86,78.73,scattered clouds,10.6112,-66.8522,
56,Hithadhoo,MV,74,84.07,overcast clouds,-0.6000,73.0833,
63,Salalah,OM,61,80.69,clear sky,17.0151,54.0924,


In [10]:
# 6a. Set parameters to search for hotels within 5000 meters
params = {
    "radius":5000,
    "type":"lodging",
    "key":g_key}

In [11]:
# 6b. Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Grab the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [12]:
len(hotel_df)

185

In [13]:
# 7. Drop rows where there is no hotel name
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] !=""]

clean_hotel_df.head(10)

,City,Country,Humidity,Max Temp,Description,Lat,Lng,Hotel Name
11,Victoria,HK,95,78.85,moderate rain,22.2855,114.1577,Mini Central
14,Oranjestad,AW,78,82.35,few clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
18,La Asuncion,VE,74,80.33,broken clouds,11.0333,-63.8628,Colinas de Matasiete
30,Georgetown,MY,89,75.15,light rain,5.4112,100.3354,Cititel Penang
34,Grand Gaube,MU,80,75.38,scattered clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
40,Mahebourg,MU,94,75.25,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
49,Caraballeda,VE,86,78.73,scattered clouds,10.6112,-66.8522,Hotel 15 Letras
56,Hithadhoo,MV,74,84.07,overcast clouds,-0.6000,73.0833,Scoop Guest House
63,Salalah,OM,61,80.69,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
71,Jacqueville,CI,83,82.08,overcast clouds,5.2052,-4.4146,Hotel Le Martin Pecheur


In [14]:
# 8a. Create the output file (CSV)
output_data_file = "WeatherPy_vacation.csv"

In [15]:
# 8b. Export the city_data into a csv
hotel_df.to_csv(output_data_file, index_label = "City ID")

In [16]:
# 9 Using the template add icty name, the country code, the weather description and maximum temperature for the city
info_box_template = """"
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the lat and long from feach row and store it in a new dataframe
locations = clean_hotel_df[["Lat","Lng"]]


In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))